In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import keras.models
from keras.models import Sequential

In [ ]:
#load data
loadname='./data_for_ML/WSS_sample 1.txt'
a1 = np.loadtxt(loadname)
loadname='./data_for_ML/WSS_sample 2.txt'
a2 = np.loadtxt(loadname)
loadname='./data_for_ML/WSS_sample 3.txt'
a3 = np.loadtxt(loadname)
loadname='./data_for_ML/WSS_sample 4.txt'
a4 = np.loadtxt(loadname)
loadname='./data_for_ML/WSS_sample 5.txt'
a5 = np.loadtxt(loadname)
loadname='./data_for_ML/WSS_sample 6.txt'
a6 = np.loadtxt(loadname)
#loadname='./data_for_ML/WSS_sample 7_non_newtonian1.txt' # since for one case we consider non-newtonian data
loadname='./data_for_ML/WSS_sample 7.txt'
a7 = np.loadtxt(loadname)

print(np.shape(a4))

In [ ]:
data = np.vstack((a1,a2,a3,a4,a5,a6,a7))
data_1 = data[:,6:9]
data_train = np.vstack((a2,a3,a4,a5,a6,a7))
data_train_1 = data_train[:,6:9]
data_test = a1
print(np.shape(data_train))
print(np.shape(data_test))

In [ ]:
data_train_p = data_train[data_train[:,-1]==1,:]#data from pruned vessels, p=pruned
data_train_up = data_train[data_train[:,-1]==0,:]#data from unpruned vessels, up=unpruned
print(np.shape(data_train_p))
print(np.shape(data_train_up))
print(data_train_up[1000,:])

In [ ]:
#data for training 0-2:coordinate data; 3-5:cylinder coordinate data; 
#6:local WSS(alphat); 7:vessel radius; 8:pressure drop; 9: scalar(not correct); 10: prune or not
x_train_p = data_train_p[:,6:9]  #x is input
x_train_up = data_train_up[:,6:9]
x_test = data_test[:,6:9]

y_train_p = data_train_p[:,10:11]  # y is output
y_train_up = data_train_up[:,10:11]
y_test = data_test[:,10:11]

#normalization
x_normal=abs(x_train_p)
x_normal=x_normal.max(axis=0)
x_train_p = x_train_p/x_normal
x_train_up = x_train_up/x_normal
x_test = x_test/x_normal

print(np.shape(x_train_p))
print(np.shape(y_train_p))
print(x_normal.max(axis=0))
print(x_train_up.max(axis=0))
print(x_train_up.min(axis=0))
print(x_train_p.max(axis=0))
print(x_train_p.min(axis=0))
print(x_test.max(axis=0))
print(x_test.min(axis=0))

In [ ]:
index = 4000

print(x_train_up[index,:])
print(y_train_up[index,:])

In [ ]:
print(y_test.max(axis=0))

In [ ]:
rand_arr = np.arange(x_train_up.shape[0])
 
np.random.shuffle(rand_arr)
print(x_train_up[rand_arr[0:5]])
print(y_train_up[rand_arr[0:5]])
print(data_train_up[rand_arr[0:5]])
print(y_train_p.max(axis=0))

In [ ]:
def simple_linear_model(input_dim):
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(units=1, input_dim=input_dim, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
input_dim = 3  # input features
nn = simple_linear_model(input_dim)

In [ ]:
lenth_pru=len(data_train_p)
print(lenth_pru)

In [ ]:
#training
loss_values=np.array([0])
vali_loss=np.array([0])
for i in range(200):
    print(i)
    rand_arr = np.arange(x_train_up.shape[0]) 
    np.random.shuffle(rand_arr)
    ran_x_train_up = x_train_up[rand_arr[0:lenth_pru]]
    ran_y_train_up = y_train_up[rand_arr[0:lenth_pru]]
    x_train = np.vstack((ran_x_train_up,x_train_p))
    y_train = np.vstack((ran_y_train_up,y_train_p))
    cost = nn.fit(x_train, y_train, epochs=10, batch_size=60000, validation_data=(x_test, y_test), verbose=1)
    loss_values = np.hstack((loss_values,cost.history['loss']))
    vali_loss = np.hstack((vali_loss,cost.history['val_loss']))

In [ ]:
# weights and bias
weights, bias = nn.layers[0].get_weights()

# Flatten weights to 1D array and format each element to 8 decimal places
formatted_weights = ["{:.8f}".format(w) for w in weights.flatten()]
formatted_bias = "{:.8f}".format(bias[0])

# Print weights and bias
print("Weights (coefficients):", formatted_weights)
print("Bias (intercept):", formatted_bias)

In [ ]:
#plot loss function
fig,ax = plt.subplots()
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
epochs = range(1, len(loss_values)+1)
plt.plot(epochs, loss_values, label='Training Loss',linewidth=2.0)
plt.plot(epochs, vali_loss, label='Test Loss',linewidth=2.0)
plt.rcParams['font.size'] = 18
plt.xlabel('Epochs',fontsize=16, fontweight='bold')
plt.ylabel('Loss',fontsize=16, fontweight='bold')
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(1.5)
plt.xlim(1, epochs[-1])
plt.ylim(0, 1)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.legend()
linewidth=10
plt.show()

In [ ]:
print(loss_values[-1])
print(vali_loss[-1])

In [ ]:
y_pred = nn.predict(x_test)
print(np.shape(y_pred))
print(y_pred.max(axis=0))
print(y_pred.min(axis=0))

In [ ]:
index = 4
print(y_pred[index])
print(y_test[index])

In [ ]:
output
np.savetxt('output_a1.txt',y_pred)